In [1]:
from PIL import Image

In [2]:
import cv2
import numpy as np
import random

# Transformation functions (from the previous example)
def add_dust(image, intensity=30, count=100):
    overlay = image.copy()
    height, width, _ = image.shape
    mask = np.zeros((height, width), dtype=np.uint8)
    for _ in range(count):
        x, y = random.randint(0, width - 1), random.randint(0, height - 1)
        size = random.randint(1, 5)
        color = (255,)  # Mask value for dust
        cv2.circle(mask, (x, y), size, color[0], -1)
        cv2.circle(overlay, (x, y), size, (random.randint(50, 150),) * 3, -1)
    return cv2.addWeighted(overlay, intensity / 100, image, 1 - intensity / 100, 0), mask


def add_smudges(image, intensity=40, count=10):
    overlay = image.copy()
    height, width, _ = image.shape
    mask = np.zeros((height, width), dtype=np.uint8)
    for _ in range(count):
        x, y = random.randint(0, width - 1), random.randint(0, height - 1)
        size_x, size_y = random.randint(20, 50), random.randint(10, 30)
        smudge_color = (255,)  # Mask value for smudges
        cv2.ellipse(mask, (x, y), (size_x, size_y), 0, 0, 360, smudge_color[0], -1)
        smudge = np.zeros_like(image)
        cv2.ellipse(smudge, (x, y), (size_x, size_y), 0, 0, 360, (random.randint(40, 100),) * 3, -1)
        smudge = cv2.GaussianBlur(smudge, (15, 15), 10)
        overlay = cv2.addWeighted(overlay, 1, smudge, intensity / 100, 0)
    return overlay, mask


def add_rain(image, intensity=50, count=100):
    overlay = image.copy()
    height, width, _ = image.shape
    mask = np.zeros((height, width), dtype=np.uint8)
    for _ in range(count):
        x_start, y_start = random.randint(0, width - 1), random.randint(0, height - 1)
        length = random.randint(15, 50)
        angle = random.uniform(-20, 20)
        x_end = int(x_start + length * np.sin(np.radians(angle)))
        y_end = int(y_start + length * np.cos(np.radians(angle)))
        color = (255,)  # Mask value for rain streaks
        thickness = random.randint(1, 2)
        cv2.line(mask, (x_start, y_start), (x_end, y_end), color[0], thickness)
        cv2.line(overlay, (x_start, y_start), (x_end, y_end), (random.randint(200, 255),) * 3, thickness)
    return cv2.addWeighted(overlay, intensity / 100, image, 1 - intensity / 100, 0), mask


def add_droplets(image, intensity=50, count=50):
    overlay = image.copy()
    height, width, _ = image.shape
    mask = np.zeros((height, width), dtype=np.uint8)
    for _ in range(count):
        x, y = random.randint(0, width - 1), random.randint(0, height - 1)
        size = random.randint(10, 30)
        droplet = np.zeros_like(image)
        cv2.circle(droplet, (x, y), size, (255, 255, 255), -1)
        cv2.circle(mask, (x, y), size, 255, -1)
        droplet = cv2.GaussianBlur(droplet, (25, 25), 10)
        overlay = cv2.addWeighted(overlay, 1, droplet, intensity / 100, 0)
    return overlay, mask

In [3]:
def add_overexposure(image, intensity=80, count=2):
    overlay = image.copy()
    height, width, _ = image.shape
    mask = np.zeros((height, width), dtype=np.uint8)

    for _ in range(count):
        shape_type = random.choice(["rectangle", "circle", "ellipse"])
        overexposure_color = (255, 255, 255)  # Bright white
        overexposure_mask_color = 255         # Mask value for overexposure

        if shape_type == "rectangle":
            x1, y1 = random.randint(0, width // 2), random.randint(0, height // 2)
            x2, y2 = random.randint(width // 2, width), random.randint(height // 2, height)
            cv2.rectangle(mask, (x1, y1), (x2, y2), overexposure_mask_color, -1)
            cv2.rectangle(overlay, (x1, y1), (x2, y2), overexposure_color, -1)
        elif shape_type == "circle":
            center = (random.randint(0, width), random.randint(0, height))
            radius = random.randint(10, min(width, height) // 4)
            cv2.circle(mask, center, radius, overexposure_mask_color, -1)
            cv2.circle(overlay, center, radius, overexposure_color, -1)
        elif shape_type == "ellipse":
            center = (random.randint(0, width), random.randint(0, height))
            axes = (random.randint(10, width // 4), random.randint(10, height // 4))
            angle = random.randint(0, 360)
            cv2.ellipse(mask, center, axes, angle, 0, 360, overexposure_mask_color, -1)
            cv2.ellipse(overlay, center, axes, angle, 0, 360, overexposure_color, -1)

    return cv2.addWeighted(overlay, intensity / 100, image, 1 - intensity / 100, 0), mask
def add_shadows(image, intensity=50, count=3):
    overlay = image.copy()
    height, width, _ = image.shape
    mask = np.zeros((height, width), dtype=np.uint8)

    for _ in range(count):
        shape_type = random.choice(["rectangle", "circle", "ellipse"])
        shadow_color = (0, 0, 0)  # Black shadow
        shadow_mask_color = 255    # Mask value for shadow

        if shape_type == "rectangle":
            x1, y1 = random.randint(0, width // 2), random.randint(0, height // 2)
            x2, y2 = random.randint(width // 2, width), random.randint(height // 2, height)
            cv2.rectangle(mask, (x1, y1), (x2, y2), shadow_mask_color, -1)
            cv2.rectangle(overlay, (x1, y1), (x2, y2), shadow_color, -1)
        elif shape_type == "circle":
            center = (random.randint(0, width), random.randint(0, height))
            radius = random.randint(10, min(width, height) // 4)
            cv2.circle(mask, center, radius, shadow_mask_color, -1)
            cv2.circle(overlay, center, radius, shadow_color, -1)
        elif shape_type == "ellipse":
            center = (random.randint(0, width), random.randint(0, height))
            axes = (random.randint(10, width // 4), random.randint(10, height // 4))
            angle = random.randint(0, 360)
            cv2.ellipse(mask, center, axes, angle, 0, 360, shadow_mask_color, -1)
            cv2.ellipse(overlay, center, axes, angle, 0, 360, shadow_color, -1)

    return cv2.addWeighted(overlay, intensity / 100, image, 1 - intensity / 100, 0), mask


In [ ]:
import os
import cv2
import numpy as np
import random




# Augmentation functions mapping

AUGMENTATIONS = {
    "dust": add_dust,
    "smudges": add_smudges,
    "rain": add_rain,
    "droplets": add_droplets,
    "shadows": add_shadows,
    "overexposure": add_overexposure,
}

AUGMENTATIONS_CLASS = {
    "dust": 1,
    "smudges": 2,
    "rain": 3,
    "droplets": 4,
    "shadows": 5,
    "overexposure": 6,

}


# Main function to apply transformations and save masks


In [5]:
from tqdm import tqdm
def process_directory_with_existing_masks(input_dir, mask_dir, output_dir, n=2):
    os.makedirs(output_dir, exist_ok=True)
    augmentations = list(AUGMENTATIONS.keys())

    for filename in tqdm(os.listdir(input_dir)):
        
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            file_path = os.path.join(input_dir, filename)
            mask_path = os.path.join(mask_dir, filename.replace("jpg", "png"))
            
            # Load image and mask
            
            image = cv2.imread(file_path)
            if image is None:
                continue
            
            existing_mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
            if existing_mask is None:
                existing_mask = np.zeros(image.shape[:2], dtype=np.uint8)

            augmented_image = image.copy()
            combined_mask = existing_mask.copy()

            for _ in range(n):
                augmentation_name = random.choice(augmentations)
                augment_fn = AUGMENTATIONS[augmentation_name]
                
                intensity = np.random.randint(30, 40)
                count = np.random.randint(50, 60)
                augmented_image, mask = augment_fn(augmented_image, count=count , intensity=intensity)
                

                # Blend new mask with existing mask using unique values for augmentation
                combined_mask[mask > 0] = AUGMENTATIONS_CLASS[augmentation_name]

            base_name = os.path.splitext(filename)[0]
            add = str(hash(str(combined_mask.tobytes())))[:6]
            augmented_path = os.path.join(output_dir, f"images/{base_name}.{add}.png")
            new_mask_path = os.path.join(output_dir, f"masks/{base_name}.{add}.png")

            # Save the final augmented image and blended mask
            cv2.imwrite(augmented_path, augmented_image)
            cv2.imwrite(new_mask_path, combined_mask)
            # print(f"Saved: {augmented_path}, {new_mask_path}")



In [9]:
!rm -r synt_data/images/
!rm -r synt_data/masks/
!mkdir synt_data/masks
!mkdir synt_data/images


In [10]:
# Example usage

input_directory = "/Users/solyanikvarvara/Desktop/misis_chill/new_photos/cv_open_dataset/open_img"  # Path to input directory
output_directory = "/Users/solyanikvarvara/Desktop/misis_chill/synt_data"  # Path to output directory
# augmentation = "dust"  # Choose from: 'dust', 'smudges', 'rain', 'droplets'
mask_directory = "/Users/solyanikvarvara/Desktop/misis_chill/new_photos/cv_open_dataset/open_msk"
n_transforms = 1
for i in range(5):
    process_directory_with_existing_masks(input_directory, mask_directory, output_directory, n=n_transforms)

100%|██████████| 250/250 [00:28<00:00,  8.74it/s]
